## Test Strategies
The main concept in hypothesis testing is test strategies. These are functions that generate stochastic input for test cases. Hypothesis maintains a test case db locally and remembers the inputs that failed the test and re-runs them. Hypothesis proives out-of-the-box strategies for almost all Python built-in types like binary, booleans, characters, integers, etc. Here is a [full list of OOB strategies](https://hypothesis.readthedocs.io/en/latest/data.html#core-strategies). A lot of strategies are parameterized, e.g., the integer strategy takes in `min` and `max`. The search strategy used to generate the input is not entirely random. In general the framework will try ~100 inputs for each test case. If there are limitations placed on the input values using the [`assume` function](https://hypothesis.readthedocs.io/en/latest/data.html#core-strategies), then the framework will try around 200 inputs to find the ones that satisfy the constraints. Upon failure, Hypothesis will find the simplest similar test input that fails and report on that. This process of finding the simplest failure example is called **shrinking**.

All test functions are to be decorated with `@given` decorator which is a parameterized decorator that takes in a strategy corresponding to each input to the test case. It accepts both positiional as well named arguments.

Another useful decorator is `@example`. This is another parameterized decorator saying that in addition to the stochastic input generated by the strategies, the input given as `@example` must be tested. A typical scenario for this is when the stochastic test strategy found an edge case on my computer, it will remember to run that test case every time I run the tests locally, my team mates might not hit this test case in their stochastic input.

The `note` function can be used as a debugging tool because it acts as a conditional print. Only prints the note if the test has failed.

If I use this with `pytest` then there are a whole bunch of useful stats that I can get. But this is not available with `unittest`.

The `@settings` attribute controls the various [settings](https://hypothesis.readthedocs.io/en/latest/settings.html) for Hypothesis and can be placed either before or after `@given`. Typically this is used to set the maximum number of examples to try, the verbosity, etc.


In [18]:
from hypothesis import given, example, note
import hypothesis.strategies as st
import unittest

In [21]:
class SomeSampleTests(unittest.TestCase):
    @given(st.integers(), st.integers())
    def test_ints_are_commutative(self, x, y):
        assert x + y == y + x

    @given(x=st.integers(), y=st.integers())
    def test_ints_cancel(self, x, y):
        assert (x + y) - y == x
    
    @given(st.lists(st.integers()))
    def test_reversing_twice_gives_same_list(slef, xs):
        # xs is a list of integers of arbritary length, usually between 0 and 100.
        ys = list(xs)
        ys.reverse()
        note(f"Input list: {xs}")
        # ys.reverse()
        assert xs == ys

    @given(st.tuples(st.booleans(), st.text()))
    def test_look_tuples_work_too(self, tpl):
        assert len(tpl) == 2
        assert isinstance(tpl[0], bool)
        assert isinstance(tpl[1], str)

In [22]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_ints_are_commutative (__main__.SomeSampleTests) ... ok
test_ints_cancel (__main__.SomeSampleTests) ... ok
test_look_tuples_work_too (__main__.SomeSampleTests) ... ok
test_reversing_twice_gives_same_list (__main__.SomeSampleTests) ... FAIL
test_decode_inverts_encode (__main__.TestEncoding) ... Falsifying example: test_reversing_twice_gives_same_list(
    slef=<__main__.SomeSampleTests testMethod=test_reversing_twice_gives_same_list>,
    xs=[0, 1],
)
Input list: [0, 1]
ok

FAIL: test_reversing_twice_gives_same_list (__main__.SomeSampleTests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-21-8774d0edcbac>", line 11, in test_reversing_twice_gives_same_list
    def test_reversing_twice_gives_same_list(slef, xs):
  File "/Users/avilay/opt/anaconda3/envs/ai/lib/python3.7/site-packages/hypothesis/core.py", line 1162, in wrapped_test
    raise the_error_hypothesis_found
  File "<ipython-input-21-8774d0edcbac